In [1]:
import re
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def parse_log(calvin_path,alg):
    with open(calvin_path) as f:
        lines = f.readlines()

    useful_lines = [line for line in lines if "txns/sec" in line or "|Config|" in line]

    first = True
    results = []
    configs = []

    for line in useful_lines:
        if "|Config|" in line:
            if not first:
                results.append(result)
            else:
                first = False
            configs.append(line)
            result = []
            continue
        result.append(line)
    results.append(result)

    l_nnodes = []
    l_dist = []
    l_WH = []
    l_tp = []
    for idx in range(len(configs)):
        config = configs[idx].strip().split("|")
        l_nnodes.append(int(config[2].split(":")[1]))
        l_dist.append(int(config[3].split(":")[1]))
        l_WH.append(int(config[4].split(":")[1]))
        run = results[idx]
        nodes=[]
        txns=[]
        if len(run) > 0:
            for idy in range(len(run)):
                line = run[idy]
                txn = line.split(": ")[1]
                txn = re.findall('(?<=Completed )([0-9]+\.?[0-9]*)',txn)
                if txn:
                    txns.append(txn[0])
                    nodes.append(line.split(": ")[0])
            txns = [float(txn) for txn in txns]
            data = {'node':nodes,'txns':txns}
            df = pd.DataFrame(data,columns=['node','txns'])
            if df.txns.gt(0).any():
                df = df.iloc[df.index[df.txns>0].tolist()[0]:]
                tp = df.groupby('node').mean().sum()[0]
            else:
                tp = 0
        else:
            tp = -1
        l_tp.append(tp)

    df_all = pd.DataFrame.from_dict({'nnodes':l_nnodes,'#WH':l_WH,'dist':l_dist,'tp':l_tp})
    df_all['alg'] = alg

    return df_all

In [3]:
df1_calvin = parse_log("./round1/calvin/typescript_calvin_bak",'Calvin')
df2_calvin = parse_log("./round2/calvin/typescript_calvin_run2",'Calvin')
df3_calvin = parse_log("./round3/calvin/typescript",'Calvin')
df1_2pl = parse_log("./round1/calvin/typescript_2pl_bak",'2pl')
df2_2pl = parse_log("./round2/calvin/typescript_2pl_run2",'2pl')
df3_2pl = parse_log("./round3/calvin/typescript_2pl_run3",'2pl')

In [ ]:
fname = ""
df = parse_log(fname,'Calvin')
# df = df = parse_log(fname,'2pl')
outname = "out.csv"
df.to_csv(outname,index=False)